In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii

#### Miscellaneous functions

In [2]:
def flatten(array): # when np.flatten() doesn't work
    temp = []
    for sublist in array:
        for item in sublist:
            temp.append(item)
    return np.array(temp)
  
def rmv(array,out): # removing galaxies in "out" from galaxies in "array"
    test_out =[] 
    for i in array:
        if i in out:
            test_out.append(False)
        else:
            test_out.append(True)
           
    return np.array(array)[test_out]


In [3]:
data = Table.read('data_all.txt', format = 'ascii')


### V^2 classification

In [47]:
galzones_old = Table.read('old_data/old/DR7_galzones.dat', format = 'ascii') 
galzones = Table.read('DR7_galzones.dat', format = 'ascii') 
zonevoids = Table.read('DR7_zonevoids.dat', format = 'ascii')
total_glx = len(galzones)

In [49]:
gal_indx = galzones_old['gal']


In [5]:
print(total_glx)

707817


In [6]:
print(len(data))

707817


#### edge and outside galaxies

In [50]:
edge_bool = [bool(x) for x in np.array(np.array(galzones['edge']))] 
edge_gal = np.array(gal_indx[edge_bool])

In [52]:
out_bool = [bool(x) for x in np.array(np.array(galzones['out']))]
out_gal = np.array(gal_indx[out_bool])

In [53]:
len(out_gal)+len(edge_gal)

375542

#### void galaxies

In [54]:
void_zones = zonevoids[(zonevoids['void0'] != -1) & (zonevoids['void1'] != -1)]['zone'] # zones that are in voids

In [55]:
void_gal_ = []
for i in void_zones:
    void_gal_.append(list(gal_indx[galzones['zone'] == i]))

void_gal_flat = flatten(void_gal_) # around 200,000 are void galaxies

In [56]:
void_gal_flat[199377] # galaxy thats outside that is in zone that is in void

328267

#### Fact #1: edge and outside galaxies exist in zones that are in voids

In [57]:
void_gal = rmv(void_gal_flat,edge_gal) # eliminating edge galazies, True for non edge, about 8000

In [58]:
void_gal = rmv(void_gal,out_gal)

In [59]:
np.intersect1d(void_gal,out_gal , return_indices=True) 

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

#### wall galaxies

In [60]:
non_void_zones = zonevoids[(zonevoids['void0'] == -1)]['zone'] # zones that are not in voids

non_void_gal = []
for i in non_void_zones:
    non_void_gal.append(list(gal_indx[galzones['zone'] == i]))
        
wall_gal = rmv(flatten(non_void_gal),edge_gal)

In [61]:
np.intersect1d(wall_gal,edge_gal , return_indices=True) 

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [62]:
wall_gal = rmv(wall_gal,out_gal)

### Check

In [63]:
len(wall_gal)+len(void_gal)+len(edge_gal)+len(out_gal), total_glx

(707817, 707817)

In [64]:
wall_gal

array([314760, 315352, 315353, ...,  17725,  16020,  44490])

### Voidfinder classification

In [21]:
data_voidfinder = Table.read('ALL_vflag_comoving.txt', format = 'ascii')

In [22]:
wall_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 0)
void_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 1)

In [23]:
out_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 9)
edge_gal_vf = np.argwhere(np.array(data_voidfinder['vflag']) == 2)

### Check

In [24]:
len(out_gal_vf)+len(edge_gal_vf)+len(wall_gal_vf)+len(void_gal_vf) == len(data_voidfinder)

True

### add flag columns and save file

In [65]:
flag_vsquared=[] # checks all galaxies in each statement as a double check for any overlaps
for i in range(len(data)):
    if i in wall_gal:
        flag_vsquared.append(0)
    if i in void_gal:
        flag_vsquared.append(1)
    if i in edge_gal:
        flag_vsquared.append(2)
    if i in out_gal:
        flag_vsquared.append(9)
        
# flag_vsquared_vf=[]
# for i in range(len(data)):
#     if i in wall_gal_vf:
#         flag_vsquared_vf.append(0)
#     if i in void_gal_vf:
#         flag_vsquared_vf.append(1)
#     if i in edge_gal_vf:
#         flag_vsquared_vf.append(2)
#     if i in out_gal_vf:
#         flag_vsquared_vf.append(9)
        
print(len(flag_vsquared),len(flag_vsquared_vf), len(data)) #check



707817 707817 707817


In [66]:
flags_table = Table([flag_vsquared_vf,flag_vsquared], names=('flag_voidfinder', 'flag_vsquared'))

data.add_columns(flags_table.columns.values())

ascii.write(data, 'data_flags_updated_Nov1.dat')  